<a href="https://colab.research.google.com/github/DIWEERAPURA/Nature-Inspired-Computing-Algorithms---Real-world-Usage/blob/main/SA_Cloud_Resource_Allocation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Upload your CSV file (aws_cloud_locations.csv)
from google.colab import files
uploaded = files.upload()  # Upload your aws_cloud_locations.csv file

import folium
import random
import math
import pandas as pd

# Load AWS Cloud Regions Dataset
dataset_path = "aws_cloud_locations.csv"  # Update if needed
df = pd.read_csv(dataset_path)
print(df)

# Extract cloud region locations (latitude and longitude)
cloud_regions = list(zip(df["Lat"], df["Long"]))  # Convert to a list of tuples

# Assign random capacities and operational costs for the regions
region_capacities = [random.randint(300, 600) for _ in range(len(cloud_regions))]
region_operational_costs = [random.randint(50, 150) for _ in range(len(cloud_regions))]

# Randomly generate user locations worldwide (latitude, longitude)
NUM_USERS = 50
users = [(random.uniform(-60, 60), random.uniform(-180, 180)) for _ in range(NUM_USERS)]

# Parameters for Simulated Annealing
INITIAL_TEMP = 1000
FINAL_TEMP = 1
ALPHA = 0.95
MAX_ITER = 100

# Cost function
def calculate_advanced_cost(region_locations, assignments):
    total_cost = 0
    region_loads = [0] * len(region_locations)

    for user_idx, region_idx in enumerate(assignments):
        latency = haversine_distance(users[user_idx], region_locations[region_idx])  # Latency as distance
        total_cost += latency  # Add latency as cost
        region_loads[region_idx] += 1  # Increment load on the assigned region

    # Add operational costs and overcapacity penalties
    for region_idx, load in enumerate(region_loads):
        total_cost += region_operational_costs[region_idx]  # Fixed operational cost
        if load > region_capacities[region_idx]:  # Overcapacity penalty
            total_cost += (load - region_capacities[region_idx]) * 50  # Penalty per extra user

    return total_cost

# Haversine distance for real-world coordinates
def haversine_distance(coord1, coord2):
    R = 6371  # Earth radius in km
    lat1, lon1 = coord1
    lat2, lon2 = coord2

    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)

    a = math.sin(dlat / 2) ** 2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c
    return distance

# Generate a random initial solution
def initialize_solution():
    assignments = [random.randint(0, len(cloud_regions) - 1) for _ in range(NUM_USERS)]
    return assignments

# Generate a neighboring solution
def generate_neighbor(current_solution):
    neighbor = current_solution[:]
    user_to_reassign = random.randint(0, NUM_USERS - 1)
    neighbor[user_to_reassign] = random.randint(0, len(cloud_regions) - 1)
    return neighbor

# Visualize the solution on a real map
def plot_solution_on_map(users, cloud_regions, solution, region_names):
    map_center = (20, 0)  # Approximate center of the world
    cloud_map = folium.Map(location=map_center, zoom_start=2)

    # Add cloud regions to the map with names
    for i, ((lat, lon), name) in enumerate(zip(cloud_regions, region_names)):
        folium.Marker(
            location=(lat, lon),
            popup=f"Cloud Region {i}: {name}",
            icon=folium.Icon(color="red"),
        ).add_to(cloud_map)

    # Add users to the map
    for i, (lat, lon) in enumerate(users):
        folium.Marker(
            location=(lat, lon),
            popup=f"User {i}",
            icon=folium.Icon(color="blue"),
        ).add_to(cloud_map)

        # Draw lines from users to assigned cloud regions
        region_idx = solution[i]
        folium.PolyLine(
            [(lat, lon), cloud_regions[region_idx]], color="gray"
        ).add_to(cloud_map)

    return cloud_map

# Simulated Annealing Algorithm
def simulated_annealing(users, cloud_regions):
    current_solution = initialize_solution()
    current_cost = calculate_advanced_cost(cloud_regions, current_solution)
    best_solution = current_solution
    best_cost = current_cost

    temperature = INITIAL_TEMP

    while temperature > FINAL_TEMP:
        for _ in range(MAX_ITER):
            neighbor_solution = generate_neighbor(current_solution)
            neighbor_cost = calculate_advanced_cost(cloud_regions, neighbor_solution)

            # Metropolis Criterion
            delta = neighbor_cost - current_cost
            if neighbor_cost < current_cost or random.random() < math.exp(-delta / temperature):
                current_solution = neighbor_solution
                current_cost = neighbor_cost

            if current_cost < best_cost:
                best_solution = current_solution
                best_cost = current_cost

        temperature *= ALPHA

    return best_solution, best_cost

best_solution, best_cost = simulated_annealing(users, cloud_regions)

# Print results
print("Best cost:", best_cost)
print("User assignments to cloud regions:", best_solution)

# Extract cloud region names from the dataset
region_names = df["Code"].tolist()  # Assuming the column "Code" contains region names

# Generate and show the map with region names
cloud_map = plot_solution_on_map(users, cloud_regions, best_solution, region_names)
cloud_map.save("cloud_allocation_map.html")
print("Map saved as 'cloud_allocation_map.html'. Open it in a browser.")


Saving aws_cloud_locations.csv to aws_cloud_locations.csv
              Code             City       Country        Lat        Long
0        us-east-2      Hillard, OH           USA  40.062924  -83.134584
1        us-east-1     Sterling, VA           USA  39.021184  -77.440525
2        us-west-1  Santa Clara, CA           USA  37.390164 -121.968163
3        us-west-2     Boardman, OR           USA  45.845729 -119.671374
4       af-south-1        Cape Town  South Africa -33.972087   18.411810
5        ap-east-1        Hong Kong     Hong Kong  22.352674  113.987616
6       ap-south-1           Mumbai         India  19.082198   72.741102
7   ap-northeast-3            Osaka         Japan  34.677623  135.416025
8   ap-northeast-2            Seoul   South Korea  37.565017  126.849467
9   ap-southeast-1        Singapore     Singapore   1.343742  103.683960
10  ap-southeast-2           Sydney     Australia -33.847927  150.651793
11  ap-northeast-1            Tokyo         Japan  35.668163  139.

In [2]:
def check_solution_accuracy(solution):
    # Calculate loads for each cloud region
    region_loads = [0] * len(cloud_regions)
    for region_idx in solution:
        region_loads[region_idx] += 1

    # Count how many regions are over capacity
    over_capacity_regions = sum(
        1 for idx, load in enumerate(region_loads) if load > region_capacities[idx]
    )

    # Calculate average latency
    latencies = [haversine_distance(users[i], cloud_regions[solution[i]]) for i in range(NUM_USERS)]
    avg_latency = sum(latencies) / NUM_USERS

    print("Average Latency (km):", avg_latency)
    print("Regions Over Capacity:", over_capacity_regions)
    print("Region Loads:", region_loads)
    print("Total Cost:", calculate_advanced_cost(cloud_regions, solution))

# Call the accuracy check function on your best solution:
check_solution_accuracy(best_solution)


Average Latency (km): 3106.3620505823865
Regions Over Capacity: 0
Region Loads: [0, 4, 6, 1, 7, 1, 1, 0, 3, 0, 7, 2, 0, 0, 3, 0, 2, 0, 0, 3, 10]
Total Cost: 157184.10252911932


In [3]:
# Calculate a baseline solution's cost using a random assignment
def calculate_baseline_cost():
    baseline_solution = initialize_solution()  # Random solution
    return calculate_advanced_cost(cloud_regions, baseline_solution)

# Compute baseline cost
baseline_cost = calculate_baseline_cost()

# Assume best_solution and best_cost were obtained from your simulated annealing algorithm
# (best_solution, best_cost) = simulated_annealing(users, cloud_regions)

# Calculate accuracy percentage as improvement over the baseline
accuracy_percentage = (baseline_cost - best_cost) / baseline_cost * 100

print("Baseline cost:", baseline_cost)
print("Optimized cost:", best_cost)
print(f"Accuracy (improvement over baseline): {accuracy_percentage:.2f}%")


Baseline cost: 484154.5862872856
Optimized cost: 157184.10252911932
Accuracy (improvement over baseline): 67.53%


In [4]:
# Upload your CSV file (aws_cloud_locations.csv)
from google.colab import files
uploaded = files.upload()  # Upload your aws_cloud_locations.csv file

import folium
import random
import math
import pandas as pd

# Load AWS Cloud Regions Dataset
dataset_path = "aws_cloud_locations.csv"  # Update if needed
df = pd.read_csv(dataset_path)
print(df)

# Extract cloud region locations (latitude and longitude)
cloud_regions = list(zip(df["Lat"], df["Long"]))  # Convert to a list of tuples

# Assign random capacities and operational costs for the regions
region_capacities = [random.randint(300, 600) for _ in range(len(cloud_regions))]
region_operational_costs = [random.randint(50, 150) for _ in range(len(cloud_regions))]

# Randomly generate user locations worldwide (latitude, longitude)
NUM_USERS = 50
users = [(random.uniform(-60, 60), random.uniform(-180, 180)) for _ in range(NUM_USERS)]

# ---------------------------
# Parameters for Simulated Annealing
# ---------------------------
# Improved parameters: higher initial temperature, slower cooling, and more iterations per temperature.
INITIAL_TEMP = 2000
FINAL_TEMP = 1
ALPHA = 0.98         # Slower cooling (closer to 1)
MAX_ITER = 200       # More iterations per temperature

# ---------------------------
# Cost Function and Haversine Distance
# ---------------------------
def haversine_distance(coord1, coord2):
    R = 6371  # Earth radius in km
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = math.sin(dlat / 2)**2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

def calculate_advanced_cost(region_locations, assignments):
    total_cost = 0
    region_loads = [0] * len(region_locations)
    # Add latency cost for each user assignment
    for user_idx, region_idx in enumerate(assignments):
        latency = haversine_distance(users[user_idx], region_locations[region_idx])
        total_cost += latency
        region_loads[region_idx] += 1
    # Add fixed operational cost and penalties for overcapacity
    for region_idx, load in enumerate(region_loads):
        total_cost += region_operational_costs[region_idx]
        if load > region_capacities[region_idx]:
            total_cost += (load - region_capacities[region_idx]) * 50
    return total_cost

# ---------------------------
# Greedy Initialization Function
# ---------------------------
def greedy_initialization():
    """Assign each user to the nearest cloud region that is below capacity if possible."""
    assignments = [-1] * NUM_USERS
    region_loads = [0] * len(cloud_regions)
    for user_idx in range(NUM_USERS):
        distances = [haversine_distance(users[user_idx], cloud_regions[i]) for i in range(len(cloud_regions))]
        sorted_indices = sorted(range(len(cloud_regions)), key=lambda i: distances[i])
        assigned = False
        for idx in sorted_indices:
            if region_loads[idx] < region_capacities[idx]:
                assignments[user_idx] = idx
                region_loads[idx] += 1
                assigned = True
                break
        if not assigned:
            # If all regions are over capacity, assign to the nearest region
            assignments[user_idx] = sorted_indices[0]
    return assignments

# ---------------------------
# Random Neighbor Generation
# ---------------------------
def initialize_solution():
    # Use the greedy solution as the starting point for better quality
    return greedy_initialization()

def generate_neighbor(current_solution):
    neighbor = current_solution[:]
    user_to_reassign = random.randint(0, NUM_USERS - 1)
    neighbor[user_to_reassign] = random.randint(0, len(cloud_regions) - 1)
    return neighbor

# ---------------------------
# Simulated Annealing Algorithm (Multiple Runs)
# ---------------------------
def simulated_annealing_run():
    current_solution = initialize_solution()
    current_cost = calculate_advanced_cost(cloud_regions, current_solution)
    best_solution = current_solution[:]
    best_cost = current_cost
    temperature = INITIAL_TEMP

    while temperature > FINAL_TEMP:
        for _ in range(MAX_ITER):
            neighbor_solution = generate_neighbor(current_solution)
            neighbor_cost = calculate_advanced_cost(cloud_regions, neighbor_solution)
            delta = neighbor_cost - current_cost
            if neighbor_cost < current_cost or random.random() < math.exp(-delta / (temperature + 1e-6)):
                current_solution = neighbor_solution
                current_cost = neighbor_cost
            if current_cost < best_cost:
                best_solution = current_solution[:]
                best_cost = current_cost
        temperature *= ALPHA
    return best_solution, best_cost

# Run multiple independent SA runs and choose the best overall solution
num_runs = 5
overall_best_cost = float("inf")
overall_best_solution = None
for i in range(num_runs):
    sol, cost = simulated_annealing_run()
    print(f"Run {i+1}: Best Cost = {cost}")
    if cost < overall_best_cost:
        overall_best_cost = cost
        overall_best_solution = sol

print("\nOverall Best Cost:", overall_best_cost)
print("Best User Assignments:", overall_best_solution)

# ---------------------------
# Visualization of the Solution on a Map
# ---------------------------
def plot_solution_on_map(users, cloud_regions, solution, region_names):
    map_center = (20, 0)  # Approximate center of the world
    cloud_map = folium.Map(location=map_center, zoom_start=2)

    # Add cloud regions to the map with names
    for i, ((lat, lon), name) in enumerate(zip(cloud_regions, region_names)):
        folium.Marker(
            location=(lat, lon),
            popup=f"Cloud Region {i}: {name}",
            icon=folium.Icon(color="red"),
        ).add_to(cloud_map)

    # Add users to the map
    for i, (lat, lon) in enumerate(users):
        folium.Marker(
            location=(lat, lon),
            popup=f"User {i}",
            icon=folium.Icon(color="blue"),
        ).add_to(cloud_map)
        # Draw line from user to assigned cloud region
        region_idx = solution[i]
        folium.PolyLine(
            [(lat, lon), cloud_regions[region_idx]], color="gray"
        ).add_to(cloud_map)
    return cloud_map

# Extract cloud region names from the dataset (assuming "Code" column contains names)
region_names = df["Code"].tolist()

# Generate and save the map
cloud_map = plot_solution_on_map(users, cloud_regions, overall_best_solution, region_names)
cloud_map.save("improved_cloud_allocation_map.html")
print("Map saved as 'improved_cloud_allocation_map.html'. Open it in a browser.")


Saving aws_cloud_locations.csv to aws_cloud_locations (1).csv
              Code             City       Country        Lat        Long
0        us-east-2      Hillard, OH           USA  40.062924  -83.134584
1        us-east-1     Sterling, VA           USA  39.021184  -77.440525
2        us-west-1  Santa Clara, CA           USA  37.390164 -121.968163
3        us-west-2     Boardman, OR           USA  45.845729 -119.671374
4       af-south-1        Cape Town  South Africa -33.972087   18.411810
5        ap-east-1        Hong Kong     Hong Kong  22.352674  113.987616
6       ap-south-1           Mumbai         India  19.082198   72.741102
7   ap-northeast-3            Osaka         Japan  34.677623  135.416025
8   ap-northeast-2            Seoul   South Korea  37.565017  126.849467
9   ap-southeast-1        Singapore     Singapore   1.343742  103.683960
10  ap-southeast-2           Sydney     Australia -33.847927  150.651793
11  ap-northeast-1            Tokyo         Japan  35.668163  

In [5]:
# Function to calculate baseline cost from a random assignment
def calculate_baseline_cost():
    baseline_solution = [random.randint(0, len(cloud_regions) - 1) for _ in range(NUM_USERS)]
    return calculate_advanced_cost(cloud_regions, baseline_solution)

# Calculate the baseline cost using a random solution
baseline_cost = calculate_baseline_cost()

# Calculate accuracy as the percentage improvement over the baseline cost
accuracy_percentage = (baseline_cost - overall_best_cost) / baseline_cost * 100

print("Baseline cost:", baseline_cost)
print("Optimized cost:", overall_best_cost)
print(f"Accuracy (improvement over baseline): {accuracy_percentage:.2f}%")


Baseline cost: 433452.5045467641
Optimized cost: 144600.5069279675
Accuracy (improvement over baseline): 66.64%


In [6]:
# Upload your CSV file (aws_cloud_locations.csv)
from google.colab import files
uploaded = files.upload()  # Upload your aws_cloud_locations.csv file

import folium
import random
import math
import pandas as pd

# Load AWS Cloud Regions Dataset
dataset_path = "aws_cloud_locations.csv"  # Update if needed
df = pd.read_csv(dataset_path)
print(df)

# Extract cloud region locations (latitude and longitude)
cloud_regions = list(zip(df["Lat"], df["Long"]))  # Convert to a list of tuples

# Assign random capacities and operational costs for the regions
region_capacities = [random.randint(300, 600) for _ in range(len(cloud_regions))]
region_operational_costs = [random.randint(50, 150) for _ in range(len(cloud_regions))]

# Randomly generate user locations worldwide (latitude, longitude)
NUM_USERS = 50
users = [(random.uniform(-60, 60), random.uniform(-180, 180)) for _ in range(NUM_USERS)]

# ---------------------------
# Parameters for Simulated Annealing (Improved)
# ---------------------------
INITIAL_TEMP = 5000       # Increased initial temperature for more exploration
FINAL_TEMP = 1
ALPHA = 0.995             # Slower cooling schedule
MAX_ITER = 500            # More iterations per temperature step

# ---------------------------
# Cost Function and Haversine Distance
# ---------------------------
def haversine_distance(coord1, coord2):
    R = 6371  # Earth radius in km
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = math.sin(dlat / 2) ** 2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

def calculate_advanced_cost(region_locations, assignments):
    total_cost = 0
    region_loads = [0] * len(region_locations)
    # Add latency cost for each user assignment
    for user_idx, region_idx in enumerate(assignments):
        latency = haversine_distance(users[user_idx], region_locations[region_idx])
        total_cost += latency
        region_loads[region_idx] += 1
    # Add fixed operational cost and penalties for overcapacity
    for region_idx, load in enumerate(region_loads):
        total_cost += region_operational_costs[region_idx]
        if load > region_capacities[region_idx]:
            total_cost += (load - region_capacities[region_idx]) * 50
    return total_cost

# ---------------------------
# Greedy Initialization Function
# ---------------------------
def greedy_initialization():
    """Assign each user to the nearest cloud region that is under capacity if possible."""
    assignments = [-1] * NUM_USERS
    region_loads = [0] * len(cloud_regions)
    for user_idx in range(NUM_USERS):
        distances = [haversine_distance(users[user_idx], cloud_regions[i]) for i in range(len(cloud_regions))]
        sorted_indices = sorted(range(len(cloud_regions)), key=lambda i: distances[i])
        assigned = False
        for idx in sorted_indices:
            if region_loads[idx] < region_capacities[idx]:
                assignments[user_idx] = idx
                region_loads[idx] += 1
                assigned = True
                break
        if not assigned:
            # If all regions are over capacity, assign to the nearest region
            assignments[user_idx] = sorted_indices[0]
    return assignments

# ---------------------------
# Neighbor Generation (Improved)
# ---------------------------
def initialize_solution():
    # Use the greedy solution as the starting point
    return greedy_initialization()

def generate_neighbor(current_solution):
    neighbor = current_solution[:]
    # With a 50% chance, reassign one user; otherwise, reassign two users
    num_changes = random.choice([1, 2])
    for _ in range(num_changes):
        user_to_reassign = random.randint(0, NUM_USERS - 1)
        neighbor[user_to_reassign] = random.randint(0, len(cloud_regions) - 1)
    return neighbor

# ---------------------------
# Simulated Annealing Algorithm (Multiple Runs)
# ---------------------------
def simulated_annealing_run():
    current_solution = initialize_solution()
    current_cost = calculate_advanced_cost(cloud_regions, current_solution)
    best_solution = current_solution[:]
    best_cost = current_cost
    temperature = INITIAL_TEMP
    while temperature > FINAL_TEMP:
        for _ in range(MAX_ITER):
            neighbor_solution = generate_neighbor(current_solution)
            neighbor_cost = calculate_advanced_cost(cloud_regions, neighbor_solution)
            delta = neighbor_cost - current_cost
            if neighbor_cost < current_cost or random.random() < math.exp(-delta / (temperature + 1e-6)):
                current_solution = neighbor_solution
                current_cost = neighbor_cost
            if current_cost < best_cost:
                best_solution = current_solution[:]
                best_cost = current_cost
        temperature *= ALPHA
    return best_solution, best_cost

# Run multiple independent SA runs and choose the best overall solution
num_runs = 10
overall_best_cost = float("inf")
overall_best_solution = None
for i in range(num_runs):
    sol, cost = simulated_annealing_run()
    print(f"Run {i+1}: Best Cost = {cost}")
    if cost < overall_best_cost:
        overall_best_cost = cost
        overall_best_solution = sol

print("\nOverall Best Cost:", overall_best_cost)
print("Best User Assignments:", overall_best_solution)

# ---------------------------
# Accuracy Calculation
# ---------------------------
def calculate_baseline_cost():
    baseline_solution = [random.randint(0, len(cloud_regions) - 1) for _ in range(NUM_USERS)]
    return calculate_advanced_cost(cloud_regions, baseline_solution)

baseline_cost = calculate_baseline_cost()
accuracy_percentage = (baseline_cost - overall_best_cost) / baseline_cost * 100
print("\nBaseline cost:", baseline_cost)
print("Optimized cost:", overall_best_cost)
print(f"Accuracy (improvement over baseline): {accuracy_percentage:.2f}%")

# ---------------------------
# Visualization of the Solution on a Map
# ---------------------------
def plot_solution_on_map(users, cloud_regions, solution, region_names):
    map_center = (20, 0)  # Approximate center of the world
    cloud_map = folium.Map(location=map_center, zoom_start=2)

    # Add cloud regions to the map with names
    for i, ((lat, lon), name) in enumerate(zip(cloud_regions, region_names)):
        folium.Marker(
            location=(lat, lon),
            popup=f"Cloud Region {i}: {name}",
            icon=folium.Icon(color="red"),
        ).add_to(cloud_map)

    # Add users to the map
    for i, (lat, lon) in enumerate(users):
        folium.Marker(
            location=(lat, lon),
            popup=f"User {i}",
            icon=folium.Icon(color="blue"),
        ).add_to(cloud_map)
        # Draw lines from users to assigned cloud regions
        region_idx = solution[i]
        folium.PolyLine(
            [(lat, lon), cloud_regions[region_idx]], color="gray"
        ).add_to(cloud_map)
    return cloud_map

# Extract cloud region names from the dataset (assuming "Code" column contains region names)
region_names = df["Code"].tolist()

# Generate and save the map with region names
cloud_map = plot_solution_on_map(users, cloud_regions, overall_best_solution, region_names)
cloud_map.save("improved_cloud_allocation_map.html")
print("\nMap saved as 'improved_cloud_allocation_map.html'. Open it in a browser.")


Saving aws_cloud_locations.csv to aws_cloud_locations (2).csv
              Code             City       Country        Lat        Long
0        us-east-2      Hillard, OH           USA  40.062924  -83.134584
1        us-east-1     Sterling, VA           USA  39.021184  -77.440525
2        us-west-1  Santa Clara, CA           USA  37.390164 -121.968163
3        us-west-2     Boardman, OR           USA  45.845729 -119.671374
4       af-south-1        Cape Town  South Africa -33.972087   18.411810
5        ap-east-1        Hong Kong     Hong Kong  22.352674  113.987616
6       ap-south-1           Mumbai         India  19.082198   72.741102
7   ap-northeast-3            Osaka         Japan  34.677623  135.416025
8   ap-northeast-2            Seoul   South Korea  37.565017  126.849467
9   ap-southeast-1        Singapore     Singapore   1.343742  103.683960
10  ap-southeast-2           Sydney     Australia -33.847927  150.651793
11  ap-northeast-1            Tokyo         Japan  35.668163  

In [7]:
# Function to compute baseline cost using a random assignment
def calculate_baseline_cost():
    baseline_solution = [random.randint(0, len(cloud_regions) - 1) for _ in range(NUM_USERS)]
    return calculate_advanced_cost(cloud_regions, baseline_solution)

# Calculate the baseline cost
baseline_cost = calculate_baseline_cost()

# Calculate accuracy percentage (improvement over baseline)
accuracy_percentage = (baseline_cost - overall_best_cost) / baseline_cost * 100

print("Baseline cost:", baseline_cost)
print("Optimized cost:", overall_best_cost)
print(f"Accuracy (improvement over baseline): {accuracy_percentage:.2f}%")


Baseline cost: 535606.8226937869
Optimized cost: 145395.97409570718
Accuracy (improvement over baseline): 72.85%
